In [172]:
#--------------------------------------------------
# Ejercicio kaggle
# STAR WARS MOVIE SCRIPT
# Rocio Radulescu ---------------------------------

import nltk as nltk
import numpy as np 
import pandas as pd
import matplotlib as plt
import seaborn as sns
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk import NaiveBayesClassifier as nbc
from nltk.classify import MaxentClassifier
from nltk.classify import DecisionTreeClassifier
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from itertools import chain
import joblib
from statistics import mode
pd.options.display.max_colwidth = 150

In [122]:
#importo csv
df_sw_ep_iv = pd.read_csv('SW_EpisodeIV.txt', delim_whitespace=True, header=None, names=["index","character","dialogue", "feeling"])
df_sw_ep_v = pd.read_csv('SW_EpisodeV.txt', delim_whitespace=True, header=None, names=["index","character","dialogue", "feeling"])
df_sw_ep_vi = pd.read_csv('SW_EpisodeVI.txt', delim_whitespace=True, header=None, names=["index","character","dialogue", "feeling"])
df_sw_ep_iv.head()

,index,character,dialogue,feeling
0,character,dialogue,NaN,NaN
1,1,THREEPIO,Did you hear that? They've shut down the main...,NaN
2,2,THREEPIO,We're doomed!,NaN
3,3,THREEPIO,There'll be no escape for the Princess this time.,NaN
4,4,THREEPIO,What's that?,NaN


In [123]:
# Divido el df para prueba  
from sklearn.model_selection import train_test_split

df_sw_ep_iv_test, df_sw_ep_iv_train = train_test_split(df_sw_ep_iv, test_size=0.02, random_state = 0)
print(df_sw_ep_iv_train.shape)
print(df_sw_ep_iv_test.shape)

(21, 4)
(990, 4)


In [173]:
# Uso primer 2% para asignar clasificacion y entrenar
sentimiento = ['pos', 'neg', 'pos', 'pos', 'neg', 'pos', 'neg', 'neg', 'pos', 'pos', 'pos', 'pos', 'neg', 'neg', 'pos', 'pos', 'pos', 'neg', 'pos', 'pos','pos'] 

# Agrego df 
df_sw_ep_iv_train['feeling'] = sentimiento 
df_sw_ep_iv_train

<ipython-input-173-f48687a874fa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sw_ep_iv_train['feeling'] = sentimiento


,index,character,dialogue,feeling
801,801,TARKIN,Yes.,pos
311,311,BEN,You don't need to see his identification.,neg
85,85,BIGGS,"I feel for you, Luke, you're going to have to learn what seems to be important or what really is important. What good is all your uncle's work if...",pos
435,435,TARKIN,"There. You see Lord Vader, she can be reasonable. Continue with the operation. You may fire when ready.",pos
204,204,LUKE,It's too dangerous with all the Sand People around. We'll have to wait until morning.,neg
590,590,HAN,And hope they don't have blasters.,pos
1,1,THREEPIO,Did you hear that? They've shut down the main reactor. We'll be destroyed for sure. This is madness!,neg
780,780,TARKIN,"You're sure the homing beacon is secure aboard their ship? I'm taking an awful risk, Vader. This had better work.",neg
457,457,HAN,"Kid, I've flown from one side of this galaxy to the other. I've seen a lot of strange stuff, but I've never seen anything to make me believe ther...",pos
299,299,LUKE,Why would Imperial troops want to slaughter Jawas?,pos


# Preparo formato del token

In [210]:
# Creo funcion para preparar el df al formato deseado para entrenar
def prepare_tokens (df_sw):

    observ = []

    #1- Creo lista con formato para entrenar
    #(consulto si es df o es row)
    if isinstance(df_sw, pd.DataFrame):
        for index, row in df_sw.iterrows():
            observ.append([row.dialogue,row.feeling])
    else:
        observ.append([df_sw.dialogue,df_sw.feeling])
        print(observ)

    #2- Se crea word_tokenize con frases de observ
    word_token = set(chain(*[word_tokenize(i[0].lower()) for i in observ]))

    #3- Elimino stopwords
    #nltk.download('stopwords')
    stopword = stopwords.words('english')
    word_token = [word for word in word_token if word not in stopword]
    
    #4- Por cada palabra-oracion agregamos booleano
    array_word = [({i:(i in word_tokenize(sentence.lower())) for i in word_token},sentiment) for sentence, sentiment in observ]

    for i in array_word:
        print(i)
        print('-----')
    
    return array_word

# Entreno con multiples algoritmos

In [186]:
# Creo funcion para entrenar clasificador segun 3 algoritmos distintos
def train_classifier (array_word):
    m_classifier = []
    # 1 - Naive bayes
    NaiveB_clasifier = nbc.train(array_word)
    m_classifier.append(NaiveB_clasifier)
    
    # 2- Max Entropy
    MaxE_classifier = MaxentClassifier.train(array_word, trace=0, max_iter=3)
    m_classifier.append(MaxE_classifier)
    
    # 3- Decision Tree
    DecisionT_clasifier = DecisionTreeClassifier.train(array_word) 
    m_classifier.append(DecisionT_clasifier)
    
    return m_classifier

# Creo función para testear algoritmos

In [190]:
# Función que combina metodos entrenados y clasificadores genericos de sentimiento

def test_clasiffier (df_words, classifier):    
    tokens = prepare_tokens(df_words)
    test_results = []
    
    # 1 - Probamos con algoritmos ya entrenados     
    for sent_tokens in tokens:
        for method in classifier:
            test_results.append(method.classify(sent_tokens[0]))
        
    # 2 - Probamos con algoritmos genericos de analisis de sentimiento
    for sentences in df_words.dialogue:

        # - VADER (0 tome como positivo)
        analyzer = SentimentIntensityAnalyzer()
        vader_result = analyzer.polarity_scores(sentences)
        if vader_result['compound']>=0 :
            test_results.append('pos')
        else: 
            test_results.append('neg')
    
        #TextBlob (0 tome como positivo)
        tb_result = TextBlob(sentences).sentiment.polarity
        if tb_result >= 0 :
            test_results.append('pos')
        else: 
            test_results.append('neg')
    
    # Devuelvo todos los resultados y la moda, o valor de mayor frecuencia
    return [test_results, mode(test_results)]

# Ejecucion de codigo

In [191]:
# 1 - Preparo tokens y entreno modelos
tokens = prepare_tokens(df_sw_ep_iv_train)
method_classifier = train_classifier(tokens)

({'lord': False, 'point': False, 'one-man': False, 'galaxy': False, '!': False, 'main': False, 'hope': False, "'re": False, 'energy': False, 'empire': False, 'fleet.its': False, 'light': False, 'greater': False, 'added': False, 'one': False, "'ve": False, 'assault': False, 'work': False, 'taking': False, 'nationalize': False, 'pull': False, 'thing': False, 'dead': False, 'dangerous': False, 'reactor': False, 'blast': False, '?': False, 'like': False, 'controlling': False, 'speed': False, 'shut': False, 'madness': False, 'force': False, 'penetrate': False, 'surely': False, 'wo': False, 'begins': False, '.': True, 'defenses': False, 'imperial': False, 'need': False, 'sure': False, 'uncle': False, 'anything': False, "'s": False, 'really': False, 'wait': False, 'people': False, 'see': False, 'know': False, 'able': False, 'star': False, 'seen': False, 'kid': False, 'identification': False, 'secure': False, 'worth': False, 'commerce': False, 'merely': False, 'fire': False, 'ship': False, 'fi

In [192]:
# 2 -Pruebo modelo con otra oracion
result = test_clasiffier(df_sw_ep_iv_test.iloc[0:1], method_classifier)
result

({'?': True, '...': True, 'consular': True, 'ambassador': True, 'ship': True}, nan)
-----


[['neg', 'pos', 'pos', 'pos', 'pos'], 'pos']

In [217]:
# 3 - Lo asigno a diversos dialogos
for index, row in df_sw_ep_iv_test.head().iterrows(): 
    result = test_clasiffier(row, method_classifier)
    df_sw_ep_iv_test.loc[index,"feeling"] = result[1]

df_sw_ep_iv_test.head(5)


[['If this is a consular ship... where is the Ambassador?', 'pos']]
({'?': True, '...': True, 'consular': True, 'ambassador': True, 'ship': True}, 'pos')
-----


HTTPError: HTTP Error 429: Too Many Requests